# Practica 3: Jaime Héctor y Álvaro Sánchez

## Preprocesamiento de datos

En primer lugar, es necesario realizar un preprocesamiento de los datos para garantizar que estén limpios, completos y en un formato adecuado que permita al modelo de machine learning aprender de manera eficiente. Para ello se deben evitar los valores nulos, normalizar las variables numéricas y codificar las variables categóricas. Además, se observa que los datos aportados en public_test.csv son demasiado grandes, lo que puede causar sobreajuste. Para evitar esto reduciremos el número de columnas desc y fgp a un número más razonable (esto también hará al modelo más veloz). Por último haremos que las columnas de los diferentes adcutos tengan más peso multiplicando por un factor de 10, ya que el tipo de aducto es esencial a la hora de predecir la collision cross section

In [51]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Cargar los datos en una variable
datos_test = pd.read_csv("public_test.csv")  # Fichero CSV con datos de test
datos_train = pd.read_csv("public_train.csv")  # Fichero CSV con datos de entrenamiento

# Separar la información del resultado o variable objetivo
x_train = datos_train.drop(columns="ccs")  # 'ccs' es la columna objetivo
y_train = datos_train["ccs"]
x_test = datos_test

# Iniciar transformadores para adaptar los datos al modelo
scaler = StandardScaler()
numeric_imputer = SimpleImputer(strategy="mean")  # Nulo numérico --> Media del resto
categorical_imputer = SimpleImputer(strategy="most_frequent")  # Nulo categórico --> Moda del resto
oh_encoder = OneHotEncoder(sparse_output=False)  # One-hot encoding: Codificar variables categóricas

# Transformador numérico (Eliminar nulos + normalizar)
numeric_transformer = Pipeline(steps=[
    ("imputer", numeric_imputer),
    ("scaler", scaler)
])

# Transformador categórico (Eliminar nulos + codificar)
categorical_transformer = Pipeline(steps=[
    ("imputer", categorical_imputer),
    ("encoder", oh_encoder)
])

# Agrupar las diferentes columnas relevantes de los datos
desc_columns = [col for col in x_train.columns if col.startswith('desc_')]
fgp_columns = [col for col in x_train.columns if col.startswith('fgp_')]
adduct_column = ["adduct"]  # 'adduct' es categórica

# Reducir la dimensionalidad de las columnas desc y fgp para que 'adduct' tenga más peso relativo
reduced_desc_columns = desc_columns[:500]  # Seleccionamos las primeras 500 columnas
reduced_fgp_columns = fgp_columns[:300]  # Seleccionamos las primeras 300 columnas

# Transformar usando ColumnTransformer, para elegir a qué columnas aplicar cada tipo de transformación
transformer = ColumnTransformer(
    transformers=[
        ("numeric", numeric_transformer, reduced_desc_columns + reduced_fgp_columns),  # Numéricos seleccionados
        ("categorical", categorical_transformer, adduct_column)  # Categóricos
    ]
)

# Aplicar los cambios realizados en los datos (no los resultados)
transformer = transformer.set_output(transform="pandas")
x_train = transformer.fit_transform(x_train)
x_test = transformer.transform(x_test)

# Aumentar el peso de las columnas del one-hot encoding de 'adduct' multiplicando por un factor para aumentar su influencia 
adduct_encoded_columns = [col for col in x_train.columns if "adduct" in col]
x_train[adduct_encoded_columns] *= 20  # Factor = 20
x_test[adduct_encoded_columns] *= 20  # Factor = 20

# Evitar la existencia de valores nulos
assert x_train.isnull().sum().sum() == 0, "Valores nulos encontrados en x_train tras el preprocesamiento"
assert x_test.isnull().sum().sum() == 0, "Valores nulos encontrados en x_test tras el preprocesamiento"


## Entrenamiento y estimación del error

Vamos a comprobar en primer lugar el error estimado al utilizar un modelo de Regresión Lineal con los datos que han sido preparados en la sección anterior.

In [58]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import median_absolute_error

# Dividir los datos: 80% entrenamiento, 20% validación
x_train_final, x_val, y_train_final, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

model1 = LinearRegression()

model1.fit(x_train_final, y_train_final)

y_val_pred1 = model1.predict(x_val)
mae_val = median_absolute_error(y_val, y_val_pred1)
    
print(f"Para Regresión lineal se obtiene una mediana de error: {mae_val}")

Para Regresión lineal se obtiene una mediana de error: 3.6065857479507315


Ahora calculamos la estimación con un Random Forest de 50 estimadores.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model2 = RandomForestRegressor(50)

model2.fit(x_train_final, y_train_final)

y_val_pred2 = model2.predict(x_val)
mae_val = median_absolute_error(y_val, y_val_pred2)
    
print(f"Para Random Forest con 50 estimadores se obtiene una mediana de error: {mae_val}")


Para Random Forest con 50 estimadores se obtiene una mediana de error: 2.771891320238069


A continuación, calculamos la estimación con un Random Forest de 100 estimadores.

In [57]:
from sklearn.ensemble import RandomForestRegressor

model3 = RandomForestRegressor(100)

model3.fit(x_train_final, y_train_final)

y_val_pred3 = model3.predict(x_val)
mae_val = median_absolute_error(y_val, y_val_pred3)
    
print(f"Para Random Forest con 100 estimadores se obtiene una mediana de error: {mae_val}")

Para Random Forest con 100 estimadores se obtiene una mediana de error: 2.804172477499918


Ya que las estimaciones anteriores han sido directamente sobre el conjuto de datos preprocesados vamos a hacer un estudio más modular. Para asegurarnos de que los resultados obtenidos son fiables utilizamos Validación Cruzada 5-Fold para el caso de Regresión Lineal y Random Forest de 50 estimadores (el de 100 estimadores no lo hacemos para ahorrar tiempo) de forma que realizaremos las estimaciones sobre diferentes subconjuntos de datos

In [ ]:
import numpy as np
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import median_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

# Dividir los datos: 70% entrenamiento, 15% validación, 15% test
x_train_final, x_temp, y_train_final, y_temp = train_test_split(x_train, y_train, test_size=0.3, random_state=42)
x_val, x_test_train, y_val, y_test_train = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

# Bucle interno de la validación cruzada
def fit_model(x_train_final, y_train_final, model):
    inner_kfold = KFold(n_splits=5, shuffle = True,random_state=42)

    for train_indices, val_indices in inner_kfold.split(x_train_final):
        x_train = x_train_final.iloc[train_indices]
        y_train = y_train_final.iloc[train_indices]
        x_val = x_train_final.iloc[val_indices]
        y_val = y_train_final.iloc[val_indices]

        model.fit(x_train, y_train)

    return model

# Modelos a comparar
models = [
    ("Linear Regression", LinearRegression()),
    ("Random Forest", RandomForestRegressor(50))
]

# Bucle externo para validación cruzada
outer_kfold = KFold(n_splits=5, shuffle = True ,random_state=42)

fold = 1
for train_val_indices, test_indices in outer_kfold.split(x_train, y_train):
    x_train_val = x_train.iloc[train_val_indices]
    y_train_val = y_train.iloc[train_val_indices]
    x_test_fold = x_train.iloc[test_indices]
    y_test_fold = y_train.iloc[test_indices]

    print(f"Fold {fold}")
    for name, model in models:
        print(f"Evaluando modelo: {name}")
        model = fit_model(x_train_val, y_train_val, model)

        y_test_pred = model.predict(x_test_fold)
        mae_test = median_absolute_error(y_test_fold, y_test_pred)
        print(f"MEDAE en el conjunto  (Fold {fold}): {mae_test}")

    fold += 1


Fold 1
Evaluando modelo: Linear Regression
MAE en el conjunto  (Fold 1): 3.6234851140346365
Evaluando modelo: Random Forest
MAE en el conjunto  (Fold 1): 2.884834199999972
Fold 2
Evaluando modelo: Linear Regression
MAE en el conjunto  (Fold 2): 3.7592405202171335
Evaluando modelo: Random Forest
MAE en el conjunto  (Fold 2): 3.024603774999875
Fold 3
Evaluando modelo: Linear Regression
MAE en el conjunto  (Fold 3): 3.5385551469926213
Evaluando modelo: Random Forest
MAE en el conjunto  (Fold 3): 2.745407020238112
Fold 4
Evaluando modelo: Linear Regression
MAE en el conjunto  (Fold 4): 3.6566890015071465
Evaluando modelo: Random Forest
MAE en el conjunto  (Fold 4): 2.780458363333352
Fold 5
Evaluando modelo: Linear Regression
MAE en el conjunto  (Fold 5): 3.3873314848351015
Evaluando modelo: Random Forest
MAE en el conjunto  (Fold 5): 2.9027976499999824


Si realizamos la media de estos valores obtenemos: 3.59% para el modelo de Regresión Lineal y 2.86% para el modelo Random Forest. Debido a que con la validación cruzada se realizan más comprobaciones con diferentes subconjuntos de datos y estamos obteniendo valores muy similares a los casos simples (3.6% y 2.77% sin validación cruzada) podemos determinar que el modelo funciona de forma equilibrada y constante, y que no presenta comportamientos distintos frente a subconjuntos de datos diferentes.

## Generación de predicciones

Como hemos visto que usando Random Forest de 50 estimadores obtenemos los mejores resultados segun la MEDAE (venciendo al modelo de regresión lineal y al random forest de 100 estimadores), utilizaremos este modelo para las predicciones finales (pero esta vez sin dividir los datos en entrenamiento y validación, sino usando directamente las columnas seleccionadas tras el preprocesamiento). 

In [62]:
#Entrenamiento con todos los datos
model2.fit(x_train, y_train)

#Predicciones del conjunto test
y_pred = model2.predict(x_test)

#Creación del csv sin cabecera
df_resultados = pd.DataFrame(y_pred)
df_resultados.to_csv('test_preds.csv', index=False, header=False)

## Conclusiones

Si analizamos el desempeño de los diferentes modelos vemos que la Regresión Lineal es un modelo funcional básico, pero presenta peor rendimiento que el Random Forest.

Se observa que el Random Forest con 50 estimadores devuelve los mejores resultados, superando incluso al modelo con 100 estimadores. Esto ocurre ya que, aunque aumentar el número de estimadores permite que el modelo pueda establecer relaciones más complejas mediante exploraciones más profundas, también aumenta el riesgo de sufrir overfitting y de adaptarse a irregularidades de los datos, lo cual empeora las predicciones futuras.

Además, al evaluar los resultados obtenidos tras utilizar validación cruzada detectamos un error medio prácticamente idéntico al de los entrenamientos iniciales, y puesto que la validación cruzada evalúa el modelo en diferentes subconjuntos de datos tratando de evitar resultados imprecisos por configuraciones concretas de los datos de entrenamiento, nos aseguramos de que efectivamente los modelos estaba funcionando de forma correcta desde un principio.

Es necesario tener en cuenta que estos resultados dependen en gran medida de los modelos utilizados y sus diferentes parámetros, así cómo de los datos utilizados y su correcto preprocesamiento para mejorar la eficiencia de los modelos.



Los porcentajes de error obtenidos (2.7% - 3.6%) son más altos de lo esperado. Esto indica que podrían realizarse más mejores complejas para reducir estos errores, aunque esto supondría un nivel de procesamiento de datos muy superior. Alguna de estas mejoras podrían ser:

-Mejor optimización de hiperparámetros realizando comprobaciones más exhaustivas en la validación cruzada

-Utilización de modelos más complejos

-Ingeniería de características más avanzada generando nuevas variables como resultado de transformaciones de las actuales tratando de tener un conjunto de datos más reducido pero igual de efectivo

-Aumentar la dimensionalidad del modelo exigiendo exploraciones más profundas o con más bucles



Además de lo documentado en este archivo, hemos tratado de emplear sistemas más complejos para abordar el problema cómo el uso de redes neuronales y el uso de estrategias cómo boosting y stacking, no obstante, de esta forma obtuvimos peores resultados (alrededor del 6%) por lo que decidimos mantener soluciones más sencillas pero aparentemente más eficaces.